<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2087, done.
remote: Counting objects: 100% (905/905), done.
remote: Compressing objects: 100% (447/447), done.
remote: Total 2087 (delta 594), reused 686 (delta 449), pack-reused 1182
Receiving objects: 100% (2087/2087), 48.49 MiB | 27.65 MiB/s, done.
Resolving deltas: 100% (1372/1372), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
140-023,980


In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
140-023,70


In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
140-023,10


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
140-023,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.0911,20.854397,74.689774,70,980,1.75,10,36.0


In [9]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


# 推薦模式

In [10]:
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

[I 2021-04-23 06:26:41,942] A new study created in memory with name: no-name-59b40042-4c8e-4d5e-9650-96dbcdad2efc


(1, 3)
(1, 41)
(1, 10)
(1, 2)
(1, 41)
(1, 10)


[I 2021-04-23 06:26:41,997] Trial 0 finished with value: 7.594423001712414 and parameters: {'Density_Feed Properties': 0.8645164493047729, 'Density_Vent Gas Production Rate and Composition': 0.43061929268938426, 'Density_Distillate Production Rate and Composition': 0.821483409376273, 'Density_Sidedraw Production Rate and Composition': 0.8812793931002879, 'Density_Bottoms Production Rate and Composition': 0.8718580659934543, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 102.9455398376683, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 12.664559947502632, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 12.449638582849303, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.8482715924822, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 191.64319796448473}. Best is trial 0 with value: 7.594423001712414.
[I 2021-04-23 06:26:42,060] Trial 1 finished with 

(1, 3)
(1, 41)
(1, 10)
bz_error: 0.0020460875460059924


[I 2021-04-23 06:27:05,636] Trial 2 finished with value: 36.15462639995857 and parameters: {'Density_Feed Properties': 0.8762702034527836, 'Density_Vent Gas Production Rate and Composition': 0.5445941795597825, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8669872836224111, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8837261533540074, 'Density_Bottoms Production Rate and Composition': 0.8707610550959779, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 119.45759689427946, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 11.683167722468704, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 10.896329165039537, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 86.52806500289638, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 89.63433752887794, 'Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw': 979.9999949809713}. B

(1, 2)
(1, 41)
(1, 10)
nainbz_error: 14.375723106347436
tol_error: 0.16788610439137663


In [11]:
bz_error,nainbz_error,tol_error

(0.0020460875460059924, 14.375723106347436, 0.16788610439137663)

# 計算delta調幅

In [12]:
c620_op2-c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
140-023,-0.001154,-0.002261,0.005736,0.001301,-0.000074,-8.066034,-0.828939,-1.089665,-1.675476,0.077673


In [13]:
c660_op2-c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
140-023,-0.000024,-0.000053,-0.000014,0.000052,0.000024,0.000003,0.000009,0.000017,-0.000005,0.000015


In [14]:
c670_op2-c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
140-023,1.788139e-07,-0.000012,2.54393,0.224062,0.194702,-0.007141,0.113937


# 檢查三項約束條件

In [15]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
140-023,69.997954


In [16]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

965.6242768936526

In [17]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

10.167886104391377